# Boundary Relation Tests

## Setup

### Imports

In [ ]:
# Shared Packages
import pandas as pd

# Local functions and classes
from types_and_classes import *
from utilities import *
from debug_tools import *
from structure_slice import *
from metrics import *
from relations import *

### Global Settings

In [60]:
PRECISION = 2


In [61]:
%matplotlib inline

## Boundary Check process
1. Build slice table (index= slice, columns = ROI, Data= StructureSlice)
2. Select Primary & Secondary ROI
	- Slice range = Min(starting slice) to Max(ending slice)
	- Primary & secondary ROI as separate series
3. Get next and previous for both ROIs using shift
	1. Identify primary slices missing a neighbour
	2. Send other slices for standard relation testing
4. Select boundaries for testing current vs current
	- Both on current slice
	- Both missing from a neighbour slice
5. Send these slices for standard relation testing and then apply boundary shift to results
6. Select boundaries for testing current vs next
	- Secondary not on current slice
	- Not overlapping on a neighbour slice
7. Identify neighbouring secondary slice that does not have a corresponding primary
8. Match the secondary neighbour with the current primary, send this for standard relation testing and then apply boundary shift to results
9. Send all remaining boundary slices for standard relation testing 
Merge all results and reduce to single relation

In [62]:
from types_and_classes import RelationshipType


def process_relations(slice_table, selected_roi) -> RelationshipType:
    mid_relations = get_non_boundary_relations(slice_table, selected_roi)
    matching_boundary_relations = get_matched_bdy_rel(slice_table, selected_roi)
    offset_boundary_relations = get_offset_bdy_rel(slice_table, selected_roi)
    all_relations = pd.concat([mid_relations,
                               matching_boundary_relations,
                               offset_boundary_relations])
    relation_binary = merge_rel(all_relations)
    return identify_relation(relation_binary)


## Test structures


### Body Structure

In [63]:
slice_spacing = 0.1
# Body structure defines slices in use
body = make_vertical_cylinder(roi_num=0, radius=10, length=1,
                                         offset_z=-0.5, spacing=slice_spacing)


### Partition


In [64]:
# Two concentric cylinders different z offsets
primary_cylinder = make_vertical_cylinder(roi_num=1, radius=2, length=0.7,
                                         offset_z=-0.3, spacing=slice_spacing)


  - Concentric cylinders starting on the same slice
  
<img src="Images\Boundaries\PartitionSup3D.png" alt="PartitionSup3D" style="height:50px;">
<img src="Images\Boundaries\PartitionSup2D.png" alt="PartitionSup2D" style="height:30px;">

In [65]:
sup_partition = make_vertical_cylinder(roi_num=2, radius=1, length=0.4,
                                         offset_z=-0.3, spacing=slice_spacing)


- Concentric cylinders ending on the same slice.

<img src="Images\Boundaries\PartitionInf3D.png" alt="PartitionInf3D" style="height:50px;">
<img src="Images\Boundaries\PartitionInf2D.png" alt="PartitionInf2D" style="height:30px;">

In [66]:
inf_partition = make_vertical_cylinder(roi_num=3, radius=1, length=0.4,
                                         offset_z=0, spacing=slice_spacing)


- Concentric cylinders starting and ending on the same slice.

<img src="Images\Boundaries\Partition3D.png" alt="PartitionInf3D" style="height:50px;">
<img src="Images\Boundaries\Partition2D.png" alt="PartitionInf2D" style="height:30px;">

In [67]:

mid_partition = make_vertical_cylinder(roi_num=4, radius=1, length=0.7,
                                         offset_z=-0.3, spacing=slice_spacing)


### Exterior Borders
  - Primary: Central Cylinder
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF cylinders in one structure with single slice gap at the SUP/INF boundary of the central cylinder.

      
![Exterior Border SUP](Images/Boundaries/ExteriorBorders2D_SUP.png)
![Exterior Border INF](Images/Boundaries/ExteriorBorders2D_INF.png)

In [68]:
# Two concentric cylinders different z offsets
outside_cylinder = make_vertical_cylinder(roi_num=5, radius=2, length=0.4,
                                         offset_z=-0.4, spacing=slice_spacing)
inside_cylinder = make_vertical_cylinder(roi_num=6, radius=1, length=0.4,
                                         offset_z=0, spacing=slice_spacing)


In [ ]:
# combine the contours
slice_data = pd.concat([body, primary_cylinder, sup_partition,
                        inf_partition, mid_partition,
                        outside_cylinder, inside_cylinder])

# convert contour slice data into a table of slices and structures
slice_table = make_slice_table(slice_data, ignore_errors=True)

In [70]:
selected_roi = [1, 2]
relation = process_relations(slice_table, selected_roi)
relation

C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.PARTITION: 7>

In [71]:
selected_roi = [1, 3]
relation = process_relations(slice_table, selected_roi)
relation


C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.PARTITION: 7>

In [72]:
selected_roi = [1, 4]
relation = process_relations(slice_table, selected_roi)
relation


C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.PARTITION: 7>

In [73]:
selected_roi = [5, 6]
relation = process_relations(slice_table, selected_roi)
relation


C:\Users\smoke\AppData\Local\Temp\ipykernel_19644\3514096742.py:5: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  all_relations = pd.concat([mid_relations,


<RelationshipType.BORDERS: 4>

# More Tests Needed

  - Single Primary slice with circular contour.
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF Cylinders in one structure with single slice gap at level of the primary slice. 
- **Partition**
  - Concentric cylinders ending on the same slice.
  - Concentric cylinders starting on the same slice
  - Primary: Central Cylinder
    - Single Secondary slice with circular contour on the SUP/INF slice of the Primary cylinder.
  - Single Primary slice with circular contour
    - Secondary: one of:
      - SUP Cylinder ending on the same slice.
      - INF Cylinder ending on the same slice.
- **CONTAINS**
  - Concentric cylinders with interior cylinder ending inside the exterior cylinder by one slice
**OVERLAPS**
  - Concentric cylinders with interior cylinder ending outside the exterior cylinder by one slice
  - Concentric cylinders with interior cylinder consisting of single slice ending inside the exterior cylinder by one slice


  - Primary: Central Cylinder
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF cylinders in one structure with single slice gap at the SUP/INF boundary of the central cylinder.
  - Single Primary slice with circular contour.
    - Secondary: one of:
      - SUP Cylinder
      - INF Cylinder
      - Combined SUP & INF Cylinders in one structure with single slice gap at level of the primary slice. 
